In [1]:
import pandas as pd
import numpy as np 
import datetime
import math
import gc
import requests
gc.collect()

from tqdm import tqdm

# Load Processed Data

In [2]:
PATH = './output/processed.csv'
df = pd.read_csv(PATH)
print(len(df))

1878292


## Challenges

There are two main challenges when weighting an EPA model.  

One, when teams get up 30 points, they will stop running efficient plays and just run clock. There needs to be some sort of penalty weight for plays when the score is out of hand. I'll try and arrive to this penalty empirically.  

Two, in the Ron Yurko et. al. paper I'm referencing, they assign weighting penalties if the next score is 4 or 5 drives in the future. Essentially, in that case, the current drive isn't deterministic when it comes to expected points. Again, I'll try to arrive to this penalty empirically.  

I'll start with the second challenge, and I'm going to approach it slightly differently. First, I'm going to take a small subset of the data and try to predict drives till next score. This should be near zero when in the opponent's redzone, and might max out (just a guess, 1.5) around a team's own 25. With multinomial logit, I can get a probability of zero, one, two, etc drives till next score. Then, using the rest of the data, I can group by drives till next score and predict probabilities of each type of score that way.

In [3]:
gb = df.groupby(['game_id','drive_id'])['down'].count().reset_index()
# gb = gb.groupby(['down'])['drive_id'].count()
# gb = gb.sort_values(ascending=False)
# gb = gb.reset_index()
# gb = gb.rename(columns={'drive_id':'play_count'})
_max = gb.down.max()
print(_max)
print(len(gb))

36
286844


In [4]:
# just for fun/validation, wanted to look at 
import seaborn as sns
import matplotlib.pyplot as plt
from scipy import stats

x = pd.Series(gb.down, name="play_count")
fig = plt.figure(figsize=[15,9])
# ax1 = sns.kdeplot(x, bw=0.14, label="play_count")
ax = sns.distplot(x, bins=np.arange(0,_max), kde=False, norm_hist=True)
ax.set(xlabel='play_count', ylabel='percentage of drives')
ax.set_title('Distribution of Play Count on 300,000 CFB Drives')
plt.show()

fig.savefig("./plots/play_counts.png")

<Figure size 1500x900 with 1 Axes>

In [5]:
# Drives labelled "FG MISSED TD" are actually made field goals, ensuing kickoff returned for TD

df.loc[df.drive_result=='FG MISSED TD', 'drive_result'] = 'FG GOOD'

In [6]:
# fix 'FG GOOD TD'
# for text in list(cat.tr_game.values):
#     print(text)
fgs = [30282005814]
rush_tds = [32252239316, 32266019408]

df.loc[df.drive_id==30282005814, 'drive_result'] = 'FG GOOD'

df.loc[df.drive_id.isin(rush_tds), 'drive_result'] = 'RUSHING TD'

In [7]:
# standardize
df.loc[df.drive_result=='FUMBLE RETURN TD', 'drive_result'] = 'FUMBLE TD'

df.loc[df.drive_result == 'INT RETURN TOUCH', 'drive_result'] = 'INT TD'

df.loc[df.drive_result == 'BLOCKED FG (TD) TD', 'drive_result'] = 'MISSED FG TD'

df.loc[df.drive_result=='POSS. ON DOWNS', 'drive_result'] = 'TURNOVER ON DOWNS'

In [8]:
# for incompletes, if there is barely any time left, drive result = end of half or end of game. 
# otherwise turnover on downs

incomp = df.loc[df.drive_result == 'INCOMPLETE']

icb = incomp.groupby(['drive_id'])['tr_game'].min().reset_index()

eofg = icb.loc[icb.tr_game < 30]
eofg_ids = list(eofg.drive_id.values)

incomp = incomp.loc[~incomp.drive_id.isin(eofg_ids)]

icb = incomp.groupby(['drive_id'])['tr_half'].min().reset_index()
eofh = icb.loc[icb.tr_half < 30]
eofh_ids = list(eofh.drive_id.values)

incomp = incomp.loc[~incomp.drive_id.isin(eofh_ids)]

down_ids = list(incomp.drive_id.values)

df.loc[df.drive_id.isin(eofg_ids), 'drive_result'] = 'END OF GAME'
df.loc[df.drive_id.isin(eofh_ids), 'drive_result'] = 'END OF HALF'
df.loc[df.drive_id.isin(down_ids), 'drive_result'] = 'TURNOVER ON DOWNS'

In [9]:
# same with completes

comp = df.loc[df.drive_result == 'PASS COMPLETE']

ccb = comp.groupby(['drive_id'])['tr_game'].min().reset_index()

eofg = ccb.loc[ccb.tr_game < 30]
eofg_ids = list(eofg.drive_id.values)

comp = comp.loc[~comp.drive_id.isin(eofg_ids)]

ccb = comp.groupby(['drive_id'])['tr_half'].min().reset_index()
eofh = ccb.loc[ccb.tr_half < 30]
eofh_ids = list(eofh.drive_id.values)

comp = comp.loc[~comp.drive_id.isin(eofh_ids)]

down_ids = list(comp.drive_id.values)

df.loc[df.drive_id.isin(eofg_ids), 'drive_result'] = 'END OF GAME'
df.loc[df.drive_id.isin(eofh_ids), 'drive_result'] = 'END OF HALF'
df.loc[df.drive_id.isin(down_ids), 'drive_result'] = 'TURNOVER ON DOWNS'

In [10]:
# same with penalties

pens = df.loc[df.drive_result == 'PENALTY']

pcb = pens.groupby(['drive_id'])['tr_game'].min().reset_index()

eofg = pcb.loc[pcb.tr_game < 30]
eofg_ids = list(eofg.drive_id.values)

pens = pens.loc[~pens.drive_id.isin(eofg_ids)]

pcb = pens.groupby(['drive_id'])['tr_half'].min().reset_index()
eofh = pcb.loc[pcb.tr_half < 30]
eofh_ids = list(eofh.drive_id.values)

pens = pens.loc[~pens.drive_id.isin(eofh_ids)]

down_ids = list(pens.drive_id.values)

df.loc[df.drive_id.isin(eofg_ids), 'drive_result'] = 'END OF GAME'
df.loc[df.drive_id.isin(eofh_ids), 'drive_result'] = 'END OF HALF'
df.loc[df.drive_id.isin(down_ids), 'drive_result'] = 'TURNOVER ON DOWNS'

In [11]:
# for drive result == kickoffs, if there is barely any time remain in half, end of half
# otherwise, i think they are just random plays out of other drives. dropping them

kos = df.loc[df.drive_result == 'KICKOFF']

kgb = kos.groupby(['drive_id'])['tr_half'].min().reset_index()

eofh = kgb.loc[kgb.tr_half <= 60]
eofh_ids = list(eofh.drive_id.values)

kos = kos.loc[~kos.drive_id.isin(eofh_ids)]
drops = list(kos.drive_id.values)

df.loc[df.drive_id.isin(eofh_ids), 'drive_result'] = 'END OF HALF'

df = df.loc[~df.drive_id.isin(drops)]

In [12]:
# drop these, they're kickoff penalties
df = df.loc[df.drive_result != 'KICKOFF RETURN TD']

In [13]:
# Rushing TD TD often is two drives with same ids

# 24318000805 Ole Miss

# 24339003820 Colorado

# 24276002413 Stanford

df.loc[((df.drive_id == 24318000805)&(df.offense=='Ole Miss')), 'drive_id'] = 2431800080500

df.loc[((df.drive_id == 24339003820)&(df.offense=='Colorado')), 'drive_id'] = 2433900382000

df.loc[((df.drive_id == 24276002413)&(df.offense=='Stanford')), 'drive_id'] = 2427600241300

df.loc[df.drive_id==2431800080500, 'drive_time'] = 150
df.loc[df.drive_id==2431800080500, 'drive_result'] = 'TURNOVER ON DOWNS'

df.loc[df.drive_id==2433900382000, 'drive_time'] = 100
df.loc[df.drive_id==2433900382000, 'drive_result'] = 'TURNOVER ON DOWNS'

df.loc[df.drive_id==2427600241300, 'drive_time'] = 100
df.loc[df.drive_id==2427600241300, 'drive_result'] = 'TURNOVER ON DOWNS'

df.loc[df.drive_result == 'RUSHING TD TD', 'drive_result'] = 'RUSHING TD'

In [14]:
gb = df.groupby(['drive_result'])['down'].count()
gb

drive_result
BLOCKED FG                 137
BLOCKED PUNT               138
BLOCKED PUNT TD            120
DOWNS                    51256
END OF GAME              34072
END OF HALF              32698
FG GOOD                 239813
FG MISSED                88936
FUMBLE                   69659
FUMBLE TD                 4028
INT                      93414
INT TD                    6712
MISSED FG TD               434
PASSING TD              158987
PASSING TD TD              153
PUNT                    619434
PUNT RETURN TD            2721
PUNT RETURN TD TD           44
PUNT TD                   1017
Punt                        48
RUSH                      1855
RUSHING TD              173503
SACK                       400
SF                        2073
TD                      212689
TIMEOUT                    148
TURNOVER ON DOWNS        73964
TURNOVER ON DOWNS TD       133
Uncategorized             9666
Name: down, dtype: int64

In [20]:
cat = df.loc[df.drive_result=='PASSING TD TD']
cat = cat.sort_values(by=['drive_id','tr_game'])
print(cat[['drive_id','offense','play_text']])

          drive_id            offense  \
1580   24248001223            Arizona   
1581   24248001223            Arizona   
1582   24248001223            Arizona   
1583   24248001223            Arizona   
1584   24248001223            Arizona   
1585   24248001223            Arizona   
1586   24248001223            Arizona   
1587   24248001223            Arizona   
13444  24255025914      Virginia Tech   
13445  24255025914      Virginia Tech   
13446  24255025914      Virginia Tech   
13447  24255025914      Virginia Tech   
13448  24255025914      Virginia Tech   
13449  24255025914      Virginia Tech   
13450  24255025914      Virginia Tech   
13451  24255025914      Virginia Tech   
13452  24255025914      Virginia Tech   
17077  24262006614         Iowa State   
17078  24262006614         Iowa State   
17079  24262006614         Iowa State   
17080  24262006614  Northern Illinois   
17081  24262006614  Northern Illinois   
17082  24262006614         Iowa State   
17083  242620066

In [21]:
# Same with PASSING TD TD (two drives with same ids)

# 24318000805 Ole Miss

# 24339003820 Colorado

# 24276002413 Stanford

df.loc[((df.drive_id == 24248001223)&(df.offense=='Northern Arizona')), 'drive_id'] = 2424800122300
df.loc[df.drive_id==2424800122300, 'drive_time'] = 100
df.loc[df.drive_id==2424800122300, 'drive_result'] = 'TURNOVER ON DOWNS'

df.loc[((df.drive_id == 24255025914)&(df.offense=='Western Michigan')), 'drive_id'] = 24255025913
df.loc[df.drive_id==24255025913, 'drive_result'] = 'BLOCKED FG'

df.loc[((df.drive_id == 24262006614)&(df.offense=='Northern Illinois')), 'drive_id'] = 2426200661400



# df.loc[df.drive_result == 'RUSHING TD TD', 'drive_result'] = 'RUSHING TD'

test = df.loc[df.drive_id == 2426200661400]
print(test.play_text)

17080         Huskies timeout; 11:09 remaining 2nd quarter
17081    A.J. Harris (NIU) rushed up the middle for 1 y...
17087    A.J. Harris (NIU) rushed up the middle for 1 y...
17088    A.J. Harris (NIU) rushed up the middle for 3 y...
17089    A.J. Harris (NIU) rushed up the middle for 4 y...
Name: play_text, dtype: object


In [17]:
for pt in list(cat.play_text.values):
    print(pt)

Kris Heavner (UA) pass left side complete to Syndric Steptoe (UA) for a 13 yard touchdown.
Mike Bell (UA) rushed right side for -3 yards.
Mike Bell (UA) rushed left side for 1 yard.
Kris Heavner (UA) pass right side complete to Mike Bell (UA) for 10 yards.
Kris Heavner (UA) rushed right side for -3 yards.
Chris Henry (UA) rushed up the middle for 7 yards.
Wildcats timeout; 08:27 remaining 4th quarter
Kris Heavner (UA) pass left side complete to Mike Bell (UA) for 11 yards.
Vince Henman (NAU) rushed up the middle for 6 yards; turnover on downs.
Jason Murrietta (NAU) pass incomplete to the left side.
Roger Robinson (NAU) rushed up the middle for 5 yards.
Jason Murrietta (NAU) pass left side complete to Raufeem Jackson (NAU) for -2 yards.
Bryan Randall (VT) pass left side complete to Eddie Royal (VT) for a 35 yard touchdown.
5 yard penalty on Broncos.
Bryan Randall (VT) pass incomplete to the right side.
Bryan Randall (VT) pass right side complete to Justin Harper (VT) for 10 yards.
Justi